In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [2]:
%load_ext autoreload
%autoreload 2 

project_dir = os.path.dirname(os.getcwd())
print('project direcotry is ', project_dir)
if project_dir not in sys.path:
    sys.path.append(project_dir)
print(sys.path)

project direcotry is  /home/yuan/bio/antibody-antigen-complex
['/home/yuan/anaconda3/envs/openfold-env/lib/python310.zip', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/lib-dynload', '', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages', '/home/yuan/anaconda3/envs/openfold-env/lib/python3.10/site-packages/openfold-2.2.0-py3.10-linux-x86_64.egg', '/home/yuan/bio/antibody-antigen-complex']


## view structure

In [3]:
# get *pdb
from src.download import Download

pdb_id = '1IC4'
pdb_file = Download.download_pdb(pdb_id)

../pdb/1IC4.pdb is ready.


In [4]:
import py3Dmol
# Create a view
view = py3Dmol.view(width=400, height=300)
view.addModel(open(pdb_file).read(), 'pdb')
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.zoomTo()
view.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## parse complex

In [5]:
# get chains
from src.parse_abag import ParseAbAg

outputs_dir = '../outputs'
p = ParseAbAg(pdb_file, outputs_dir)
# p.structure.header

# get chains
chains = p.get_chains()
pprint(chains)

Successfully retrieve structure of 1IC4
outputs dir:  ../outputs/1I/1IC4
[{'chains': [{'AA residues': 127,
              'chain_id': 'L',
              'seq': 'DIVLTQSPATLSVTPGNSVSLSCRASQSIGNNLHWYQQKSHESPRLLIKYASQSISGIPSRFSGSGSGTDFTLSINSVETEDFGMYFCQQSNSWPYTFGGGTKLEIK'},
             {'AA residues': 134,
              'chain_id': 'H',
              'seq': 'DVQLQESGPSLVKPSQTLSLTCSVTGDSITSAYWSWIRKFPGNRLEYMGYVSYSGSTYYNPSLKSRISITRDTSKNQYYLDLNSVTTEDTATYYCANWDGDYWGQGTLVTVSAA'},
             {'AA residues': 155,
              'chain_id': 'Y',
              'seq': 'KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRNTDGSTDYGILQINSRWWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWRNRCKGTDVQAWIRGCRL'}],
  'model_id': 0}]


In [6]:
p.structure.header['compound']

{'1': {'misc': '',
  'molecule': 'lysozyme binding ig kappa chain',
  'chain': 'l',
  'engineered': 'yes'},
 '2': {'misc': '',
  'molecule': 'igg1 fab chain h',
  'chain': 'h',
  'fragment': 'residues 1-114',
  'engineered': 'yes',
  'mutation': 'yes'},
 '3': {'misc': '',
  'molecule': 'lysozyme c',
  'chain': 'y',
  'ec_number': '3.2.1.17',
  'ec': '3.2.1.17'}}

In [7]:
# filter: is antibody-antigen complex
p.filter_antibody_antigen()
p.update_chains('chain_type')
print(p.structure_id)
pprint(p.chains[0]['chains'][0])

Pairwise chains of 1IC4 are updated.
1IC4
{'AA residues': 127,
 'chain_id': 'L',
 'chain_type': 'antigen',
 'seq': 'DIVLTQSPATLSVTPGNSVSLSCRASQSIGNNLHWYQQKSHESPRLLIKYASQSISGIPSRFSGSGSGTDFTLSINSVETEDFGMYFCQQSNSWPYTFGGGTKLEIK'}


In [8]:
# split chains
p.chains_to_pdb()

Saved chain ['H'] to ../outputs/1I/1IC4/1IC4_H.pdb
Saved chain ['L'] to ../outputs/1I/1IC4/1IC4_L.pdb
Saved chain ['H', 'L'] to ../outputs/1I/1IC4/1IC4_H_L.pdb
Saved chain ['Y'] to ../outputs/1I/1IC4/1IC4_Y.pdb
Saved chain ['H', 'Y'] to ../outputs/1I/1IC4/1IC4_H_Y.pdb


In [9]:
p.bounded

[{'type': 'single-antigen',
  'chains': [['H'], ['L'], ['H', 'L']],
  'pdb_files': ['../outputs/1I/1IC4/1IC4_H.pdb',
   '../outputs/1I/1IC4/1IC4_L.pdb',
   '../outputs/1I/1IC4/1IC4_H_L.pdb']},
 {'type': 'single-antigen',
  'chains': [['H'], ['Y'], ['H', 'Y']],
  'pdb_files': ['../outputs/1I/1IC4/1IC4_H.pdb',
   '../outputs/1I/1IC4/1IC4_Y.pdb',
   '../outputs/1I/1IC4/1IC4_H_Y.pdb']}]

In [10]:
p.save_annot()

https://www.ebi.ac.uk/pdbe/api/mappings/1ic4
Save annotation of 1IC4: ../outputs/1I/1IC4/meta.json


## SASA

In [11]:
# calculate SASA
sasa_file = p.build_freesasa()

In [12]:
from src.analyze_sasa import AnalyzeSasa

p2= AnalyzeSasa(sasa_file)
p2.print_meta()
print(list(p2.data[0]['delta_total_sasa']))

key1 ['type', 'chains', 'pdb_files', 'sasa_files', 'delta_total_sasa']
single chain H ~ antigen chain L
------------------------------
key1 ['type', 'chains', 'pdb_files', 'sasa_files', 'delta_total_sasa']
single chain H ~ antigen chain Y
------------------------------
['delta1', 'delta1_file', 'delta2', 'delta2_file']


In [13]:
# retrieve sequences
motifs = p2.retrieve_seq()
print('output file: ', motifs[1])
motifs

output file:  ../outputs/1I/1IC4/seq_sasa_delta-0_span-30.p


([('single',
   [{'seq': 'IRKFPGNRLEYMGYVSYSGSTYYNP',
     'sig_res': 12,
     'start': (36, '37'),
     'end': (60, '61'),
     'delta_sasa_cutoff': 0,
     'max_span': 30},
    {'seq': 'YCANWDGDYWGQ',
     'sig_res': 8,
     'start': (93, '94'),
     'end': (104, '105'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}],
   'antigen',
   [{'seq': 'D',
     'sig_res': 1,
     'start': (0, '1'),
     'end': (0, '1'),
     'delta_sasa_cutoff': 0,
     'max_span': 30},
    {'seq': 'YQQKSHESPRLLIKYASQSISGIPSRFSGSGSGTDFTLSINSVETEDFGMYFCQQSNSWPYTFGGGTK',
     'sig_res': 22,
     'start': (35, '36'),
     'end': (102, '103'),
     'delta_sasa_cutoff': 0,
     'max_span': 30}]),
  ('single',
   [{'seq': 'DSITSAYWSWIRKFPGNRLEYMGYVSYSGSTY',
     'sig_res': 11,
     'start': (26, '27'),
     'end': (57, '58'),
     'delta_sasa_cutoff': 0,
     'max_span': 30},
    {'seq': 'WDGD',
     'sig_res': 3,
     'start': (97, '98'),
     'end': (100, '101'),
     'delta_sasa_cutoff': 0,
     'max_span':

In [14]:
# dataset for ML
seq_sasa_file = motifs[1]
AnalyzeSasa(seq_sasa_file).build_dataset()

[{'input': 'D | YQQKSHESPRLLIKYASQSISGIPSRFSGSGSGTDFTLSINSVETEDFGMYFCQQSNSWPYTFGGGTK',
  'input_label': 'antigen',
  'output': 'IRKFPGNRLEYMGYVSYSGSTYYNP | YCANWDGDYWGQ',
  'output_label': 'single'},
 {'input': 'YR | WWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNA',
  'input_label': 'antigen',
  'output': 'DSITSAYWSWIRKFPGNRLEYMGYVSYSGSTY | WDGD',
  'output_label': 'single'}]

In [15]:
# complex
p2.print_delta_sasa(0)

complex: single chain H ~ antigen chain L
------------------------------
single chain:
    chain_id res_no  res aa      value
36         H     37  ILE  I   2.703881
38         H     39  LYS  K  36.936685
42         H     43  ASN  N  56.694124
43         H     44  ARG  R   2.703881
44         H     45  LEU  L  85.805441
46         H     47  TYR  Y  94.706125
48         H     49  GLY  G   5.407762
49         H     50  TYR  Y  25.670532
57         H     58  TYR  Y  32.031289
58         H     59  TYR  Y  12.432338
59         H     60  ASN  N  33.850284
60         H     61  PRO  P  32.446570
93         H     94  TYR  Y  36.971544
97         H     98  TRP  W  33.225886
98         H     99  ASP  D  16.988577
99         H    100  GLY  G  30.155369
100        H    101  ASP  D  16.223285
102        H    103  TRP  W  83.595192
103        H    104  GLY  G   6.995296
104        H    105  GLN  Q  13.402386
------------------------------
antigen chain:
    chain_id res_no  res aa       value
0       

## distance

In [16]:
# calculate distance
dist_file = p.build_dist()

In [17]:
from src.analyze_dist import AnalyzeDist

max_span = 30
p3= AnalyzeDist(dist_file, max_span=max_span)
p3.print_meta()
print(list(p3.data[0]['dist']))

key1 ['type', 'chains', 'pdb_files', 'dist']
single chain H ~ antigen chain L
------------------------------
key1 ['type', 'chains', 'pdb_files', 'dist']
single chain H ~ antigen chain Y
------------------------------
['dist1', 'dist1_file', 'dist2', 'dist2_file']


In [18]:
# retrieve sequences
motifs = p3.retrieve_seq()
motifs

([('single',
   [{'seq': 'IRKFPGNRLEYMGYVSYSGSTYYNP',
     'sig_res': 13,
     'start': (36, 37),
     'end': (60, 61),
     'dist_cutoff': 5,
     'max_span': 30},
    {'seq': 'YCANWDGDYWGQG',
     'sig_res': 9,
     'start': (93, 94),
     'end': (105, 106),
     'dist_cutoff': 5,
     'max_span': 30}],
   'antigen',
   [{'seq': 'YQQKSHESPRLLIKY',
     'sig_res': 7,
     'start': (35, 36),
     'end': (49, 50),
     'dist_cutoff': 5,
     'max_span': 30},
    {'seq': 'MYFCQQSNSWPYTFGG',
     'sig_res': 8,
     'start': (84, 85),
     'end': (99, 100),
     'dist_cutoff': 5,
     'max_span': 30}]),
  ('single',
   [{'seq': 'TSAYWSWIRKFPGNRLEYMGYVSYSGSTY',
     'sig_res': 11,
     'start': (29, 30),
     'end': (57, 58),
     'dist_cutoff': 5,
     'max_span': 30},
    {'seq': 'WD',
     'sig_res': 2,
     'start': (97, 98),
     'end': (98, 99),
     'dist_cutoff': 5,
     'max_span': 30}],
   'antigen',
   [{'seq': 'YR',
     'sig_res': 2,
     'start': (19, 20),
     'end': (20, 21)

In [19]:
seq_dist_file = motifs[1]
AnalyzeDist(seq_dist_file).build_dataset()

[{'input': 'YQQKSHESPRLLIKY | MYFCQQSNSWPYTFGG',
  'input_label': 'antigen',
  'output': 'IRKFPGNRLEYMGYVSYSGSTYYNP | YCANWDGDYWGQG',
  'output_label': 'single'},
 {'input': 'YR | WWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNA',
  'input_label': 'antigen',
  'output': 'TSAYWSWIRKFPGNRLEYMGYVSYSGSTY | WD',
  'output_label': 'single'}]

In [20]:
# complex
p3.print_min_dist(0)

------------------------------
     res_no  res atom      value aa
0         1  ASP    C  11.729742  D
1         2  VAL    C  11.407640  V
2         3  GLN    C  10.638920  Q
3         4  LEU    C   7.874747  L
4         5  GLN    C   9.238106  Q
..      ...  ...  ...        ... ..
109     110  THR    C  14.083373  T
110     111  VAL    C  18.115705  V
111     112  SER    C  20.405320  S
112     113  ALA    C  23.548512  A
113     114  ALA    C  22.769509  A

[114 rows x 5 columns]
------------------------------
    res_no  res atom     value aa
35      36  TYR    C  3.688513  Y
37      38  GLN    C  2.893987  Q
41      42  GLU    C  4.069521  E
42      43  SER    C  2.626992  S
43      44  PRO    C  3.518977  P
45      46  LEU    C  3.474888  L
49      50  TYR    C  4.766182  Y
84      85  MET    C  4.036103  M
86      87  PHE    C  3.479552  F
88      89  GLN    C  3.987524  Q
93      94  TRP    C  2.838357  W
94      95  PRO    C  3.668871  P
95      96  TYR    C  2.981909  Y
97    